In [1]:
import torch
import cv2
import numpy as np
import torch.nn as nn
from torchvision.transforms import ToTensor
import time
import pycuda.driver as cuda
import pycuda.autoinit
import tensorrt as trt

In [2]:
!ls -ltrh /dev/video*

crw-rw---- 1 root video 81, 0 Dec 21 22:40 /dev/video0


In [3]:
from jetcam.usb_camera import USBCamera

#TODO change capture_device if incorrect for your system
camera = USBCamera(width=224, height=224, capture_width=640, capture_height=480, capture_device=0)

In [4]:
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

image_widget = ipywidgets.Image(format='jpeg')

display(image_widget)

# Capture and preprocess 25 frames
captured_frames = []
current_frame_count = 0
SEQUENCE_LENGTH = 25
IMAGE_HEIGHT = 64
IMAGE_WIDTH = 64


while current_frame_count < SEQUENCE_LENGTH:
    def update_image(change):
        image = change['new']
        image_widget.value = bgr8_to_jpeg(image)
    
    camera.observe(update_image, names='value')

    image = camera.read()
    resized_frame = cv2.resize(image, (IMAGE_HEIGHT, IMAGE_WIDTH))
    normalized_frame = resized_frame / 255.0
    captured_frames.append(normalized_frame)
    current_frame_count += 1
    camera.unobserve(update_image, names='value')

Image(value=b'', format='jpeg')

In [11]:
import cv2
import numpy as np
import torch
import tensorrt as trt
from torchvision.transforms import ToTensor

CLASSES_LIST = ["WalkingWithDog", "TaiChi", "Swing", "HorseRace"]

# Load the TensorRT model
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
with open('conv3D_model_best.trt', 'rb') as f, trt.Runtime(TRT_LOGGER) as runtime:
    engine = runtime.deserialize_cuda_engine(f.read())
    context = engine.create_execution_context()

# Define the transform for input frames
transform = ToTensor()

def preprocess_video(frames):
    frames = [transform(frame.astype(np.float32)) for frame in frames]
    frames_tensor = torch.stack(frames)
    return frames_tensor.unsqueeze(0)

# Replace 'input_video_path' with the path to your input video
input_data = preprocess_video(captured_frames)


# Define the shape of the output tensor based on your model
output_shape = (1, 4)  # Adjust this based on your model's output shape

# Create an empty output tensor
output = np.empty(output_shape, dtype=np.float32)

# Allocate device memory for inputs and outputs
d_input = cuda.mem_alloc(input_data.element_size() * input_data.nelement())
d_output = cuda.mem_alloc(output.nbytes)


# Create a stream
stream = cuda.Stream()
###########################################
print("D_INPUT : ")
print(d_input)
            
print("input_data : ")
print(input_data)
            
print(input_data.shape)
############################################
# Transfer input data to device
cuda.memcpy_htod_async(d_input, input_data.numpy().ravel(), stream)

# Measure inference time
start_time = time.time()

# Execute inference
context.execute_async(bindings=[int(d_input), int(d_output)], stream_handle=stream.handle)
stream.synchronize()

end_time = time.time()
inference_time = end_time - start_time

# Transfer predictions back to host
cuda.memcpy_dtoh_async(output, d_output, stream)
stream.synchronize()

print("D_OUTPUT : ")
print(d_output)
            
print("OUTPUT : ")
print(output)
            
print(output.shape)

# Convert the NumPy array to a PyTorch tensor
output_tensor = torch.from_numpy(output)

# Get the predicted class index
_, predicted_class = torch.max(output_tensor, 1)
predicted_class_index = predicted_class.item()

# Map the index to the class name
predicted_class_name = CLASSES_LIST[predicted_class_index]

#Check 4 & 5 output raw tensor
print("Raw Model Output:", output_tensor)

print("Predicted Class Index:", predicted_class_index)

print(f"The model predicts the input video belongs to class: {predicted_class_name}")
print(f"Inference time: {inference_time:.5f} seconds")

# Release the camera
camera.running = False

D_INPUT : 
input_data : 
tensor([[[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

          [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

          [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
 

In [ ]:
import os
os._exit(00)

In [ ]:
import os

# Get the size of the serialized engine file
model_size = os.path.getsize('/nvdli-nano/data/Inference/conv3D_model.trt')

print(f"The size of the TensorRT model file is: {model_size / (1024 * 1024):.2f} MB")